In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import numpy as np

from utils.gen_bert_embedding import circRNABert, seq2kmer_bert

from transformers import BertModel, BertTokenizer

from utils.HDRNet import *
from utils.utils import *

def seq2kmer_bert(seq, k):
    seq_length = len(seq)
    import random
    kmer = [seq[x:x + k] for x in range(seq_length - k + 1)]
    kmers = " ".join(kmer)
    return kmers

In [2]:
max_length = 101

filename = 'TIA1_Hela.tsv'

name, sequences, structs, label = read_csv_with_name('dataset/' + filename)

# name = list(name)

# sequences = list(sequences)

index = [1998, 2, 13]  # Samples be plotted
name = name[index]
sequences = sequences[index]
structs = structs[index]
label = label[index]



In [3]:
structure = np.zeros((len(structs), 1, max_length))  # (N, 1, 101)
for i in range(len(structs)):
    struct = structs[i].split(',')
    ti = [float(t) for t in struct]
    ti = np.array(ti).reshape(1, -1)
    structure[i] = np.concatenate([ti], axis=0)

# Generate bert embedding
model__path = './BERT_Model/'  
tokenizer = BertTokenizer.from_pretrained(model__path, do_lower_case=False)
model = BertModel.from_pretrained(model__path)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# model = torch.nn.DataParallel(model, device_ids=[0, 1, 2, 3])
model = model.eval()


bert_embedding = circRNABert(sequences, model, tokenizer, device, 3)  # default k=3  # (N, 101, 768)
bert_embedding = bert_embedding.transpose([0, 2, 1])  # (N, 768, 101)

Some weights of the model checkpoint at ./BERT_Model/ were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
bert_embedding.shape

(3, 768, 99)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HDRNet().to(device)

# modelname = filename.replace('K562', 'HepG2')
# modelname = filename.replace('HepG2', 'K562')  # For dynamic prediction
model_file = 'results/model/' + filename.rstrip('.tsv') + '.pth'
model.load_state_dict(torch.load(model_file))
model.eval()

HDRNet(
  (conv0): Conv1d(
    (conv): Conv1d(768, 128, kernel_size=(1,), stride=(1,), bias=False)
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (conv1): Conv1d(
    (conv): Conv1d(1, 128, kernel_size=(3,), stride=(1,), bias=False)
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
  )
  (multiscale_str): multiscale(
    (conv0): Conv1d(
      (conv): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (conv1): Sequential(
      (0): Conv1d(
        (conv): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
        (relu): ReLU(inplace=True)
      )
      (1): Conv1d(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn): BatchNorm1d(32, eps=

In [6]:
prob = torch.sigmoid(model(torch.tensor(bert_embedding).to(device).float(), torch.tensor(structure).to(device).float()))
prob  # See binding predictions

tensor([[0.9990],
        [1.0000],
        [0.9979]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [7]:
import shap
import pandas as pd
torch.cuda.empty_cache()
test_emb = torch.tensor(bert_embedding).requires_grad_().to(device).type(torch.float32)
test_struc = torch.tensor(structure).requires_grad_().to(device).type(torch.float32)
# print(test_emb)
e = shap.GradientExplainer(model, [test_emb, test_struc])

i=1 # First see the 0-th sample
shap_values = e.shap_values([test_emb[i:i+1], test_struc[i:i+1]])

def norm(data):
    _range = np.max(data,axis=1) - np.min(data, axis=1)
    return (data - np.min(data,axis=1)) / _range

# sss = seq2kmer_bert(test_seq[i], 3).split(' ')

input_seq = sequences[i]
input_struct = test_struc[i]

# bert_gradient_data = norm(shap_values[0]) #[1, 768, 99]
# bert_gradient_data = np.exp(shap_values[0])

bert_attention_data = np.max(shap_values[0], axis=1)  # [1,99]  
bert_attention_data = np.insert(bert_attention_data, 0, values=0, axis=1)
bert_attention_data = np.insert(bert_attention_data, bert_attention_data.shape[-1], values=0, axis=1)
bert_attention_data = convert_one_hot2([input_seq], bert_attention_data[0,:]).transpose(1,0,2).squeeze()

struc_attention_data = shap_values[1][0]
# (shap_values[1][0]-np.min(shap_values[1][0]))/(np.max(shap_values[1][0])-np.min(shap_values[1][0]))

W = np.concatenate([bert_attention_data, struc_attention_data], axis=0)

x_str = input_struct.cpu().detach().numpy().reshape(101, 1)
str_null = np.zeros_like(x_str)
ind =np.where(x_str == -1)[0]
str_null[ind,0]=1
str_null=np.squeeze(str_null).T


onehot = convert_one_hot([input_seq]).squeeze()

X = np.concatenate([onehot, input_struct.cpu().detach().numpy()], axis=0)


# df = pd.DataFrame(data) #, columns=sss)

In [8]:
# Plot the salience map of the selected sample
import utils.visualize as visualize
visualize.plot_saliency(X, W, nt_width=100, norm_factor=3, str_null=str_null, outdir="results/high_attention_region_plot/out.png")

## Plot the salience map
![pic_salience_map](./results/high_attention_region_plot/out.png)